<a href="https://colab.research.google.com/github/scapolingua/Deep-Learning/blob/master/AddestraMyDocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#93c7da835cb17c9b4a59ccae94d32fdeaedfebc1
!pip install python-docx
!pip install pymupdf
!pip install transformers datasets torch
!pip install evaluate
#!pip install nltk

In [4]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
#93c7da835cb17c9b4a59ccae94d32fdeaedfebc1

import evaluate

import numpy as np

def compute_metrics(eval_pred):
    """
    Calcola le metriche di valutazione.

    Args:
        eval_pred: Tupla contenente le previsioni e le etichette.

    Returns:
        Dizionario contenente le metriche calcolate.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Utilizza la metrica "accuracy" di Hugging Face Evaluate
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

#Ecco come leggere il testo da un documento Word:

from docx import Document

def estrai_testo_word(file_path):
    doc = Document(file_path)
    testo = ""
    for paragrafo in doc.paragraphs:
        testo += paragrafo.text + "\n"
    return testo

# Esempio di utilizzo
testo_word = estrai_testo_word('documento.docx')
print(testo_word)


import fitz  # PyMuPDF

def estrai_testo_pdf(file_path):
    doc = fitz.open(file_path)
    testo = ""
    for pagina in doc:
        testo += pagina.get_text()
    return testo

# Esempio di utilizzo
testo_pdf = estrai_testo_pdf('documento.pdf')
print(testo_pdf)


#!pip install nltk

import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Scarica risorse necessarie di NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Funzione di preprocessing
def preprocessamento(testo):
    # Rimozione dei caratteri non alfanumerici
    testo = re.sub(r'\W+', ' ', testo.lower())
    # Tokenizzazione
    parole = word_tokenize(testo)
    # Rimozione delle stopwords
    parole = [p for p in parole if p not in stopwords.words('italian')]
    # Lemmatizzazione
    lemmatizer = WordNetLemmatizer()
    parole = [lemmatizer.lemmatize(p) for p in parole]
    return ' '.join(parole)

# Esempio di utilizzo
testo_preprocessato = preprocessamento(testo_pdf)

print(testo_preprocessato)


from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from transformers import AutoTokenizer, GPT2Model

from datasets import Dataset

# Carica il modello pre-addestrato e il tokenizer
model_name = "gpt2"  # Puoi usare "gpt2-medium" o "gpt2-large" per modelli più grandi
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

#tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
#model = TFAutoModel.from_pretrained(pretrained_weights)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model.resize_token_embeddings(len(tokenizer))

from nltk.tokenize import sent_tokenize

#sentences = sent_tokenize(testo_preprocessato)

#dataset = Dataset.from_dict({"text": sentences})  # Usa il tuo dataset qui

# Prepara i tuoi dati (testo preprocessato)
dataset = Dataset.from_dict({"text": [testo_preprocessato]})  # Usa il tuo dataset qui

# Tokenizzazione del dataset
'''
def tokenizza(x):
    return tokenizer(x['text'], truncation=True, padding="max_length", max_length=512,return_tensors="pt")
'''

def tokenizza(x):
    # Tokenize the text and create labels (targets)
    tokenized_inputs = tokenizer(x['text'], truncation=True, padding="max_length", max_length=512)
    # Create labels by shifting the input ids to the right
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Tokenizzazione del dataset
dataset = dataset.map(tokenizza, batched=True)

#dataset = dataset.train_test_split(test_size=0.2) # 80% train, 20% test


# Impostazioni per l'addestramento
training_args = TrainingArguments(
    output_dir="gpt2_finetuned",  # Cartella per salvare il modello fine-tunato
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_dir='logs',
    logging_steps=10,
    save_steps=1000,
)

# Crea il Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,            # for padding batched data
    #compute_metrics=compute_metrics # for custom metrics
)


# Avvia l'addestramento
trainer.train()



def rispondi_a_domanda(domanda):
    # Tokenizza la domanda
    input_ids = tokenizer.encode(domanda, return_tensors="pt")
    tokenizer.pad_token_id = tokenizer.eos_token_id
    # Genera la risposta
    risposta_ids = model.generate(input_ids, max_length=150, num_return_sequences=1)
    risposta = tokenizer.decode(risposta_ids[0], skip_special_tokens=True)
    return risposta

pad_token_id='eos_token_id`:50256'

# Esempio di interrogazione
domanda = "Qual è il tema principale del documento?"

print(rispondi_a_domanda(domanda))



                                                                          







Regolamento Sistema Informativo Comunale
















	
 
Indice
II Sistema Informativo Comunale	3
Definizione di hardware	3
Definizione di software	3
Sistemi operativi	3
Applicazioni per la gestione dei server	3
Applicazioni gestionali	3
Definizione di Rete Telematica Comunale	3
Definizione della struttura di gestione	4
Personale della struttura di gestione	4
Compiti della struttura di gestione	4
Definizione di operatori	4
Direttive per l'utilizzo del SIC	5
Assegnazione delle postazioni	5
Utilizzo delle postazioni	5
Utilizzo della rete comunale	6
Utilizzo di Internet	6
Utilizzo della posta elettronica	7
Salvataggio dei dati	8
Proprietà intellettuale	8
Approvvigionamenti SIC e dotazioni	8
Manutenzione SIC e dotazioni	8
Danneggiamene	9
Sanzioni	9
Aggiornamenti delle direttive	9
Allegato A - Linee guida per l'e-mail	10
Premessa	10
Nomenclatura caselle e-mail	11
Allegato B -Gestione e Funzione del sito We

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


via po 53 10124 torino italy tel 39 011 6704917 fax 39 011 6703895 url http www de unito it working paper series capitale sociale pensiero john maynard keynes damiano fiorillo dipartimento economia s cognetti de martiis centro studi storia metodi economia politica claudio napoleoni cesmep working paper no 01 2007 università torino capitale sociale pensiero john maynard keynes damiano fiorillo dipartimento scienze economiche statistiche università salerno 2 capitale sociale pensiero john maynard keynes damiano fiorillo abstract obiettivo lavoro accertare aspetti capitale sociale quali relazioni interpersonali organizzazioni sociali fiducia possono individuarsi economia politica keynesiana lavoro prova rendere contributi lungo tre linee primo filosofia keynesiana sistema capitalista individuo permeato obiettivi egoistici razionali quando instaura relazioni interpersonali tuttavia keynes rifiuta eticamente movente vantaggio contrappone massimizzazione ideale secondo individualismo egoisti

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

<ipython-input-15-a27cf6134d5d>:143: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Qual è il tema principale del documento?

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.

The following is a list of all the documents that are available in the archive.
The following is a list of all the documents that are


In [23]:
def rispondi_a_domanda(domanda):
    # Tokenizza la domanda
    input_ids = tokenizer.encode(domanda, return_tensors="pt")
    tokenizer.pad_token_id = 50256 # tokenizer.eos_token_id

    attention_mask = input_ids.ne(tokenizer.pad_token_id).int()

    # Genera la risposta
    risposta_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_return_sequences=1)
    #risposta_ids = model.generate(input_ids, max_length=150, num_return_sequences=1)
    risposta = tokenizer.decode(risposta_ids[0], skip_special_tokens=True)
    return risposta

#pad_token_id='eos_token_id`:50256'

# Esempio di interrogazione
domanda = "Qual è il tema principale del documento?"
print(rispondi_a_domanda(domanda))


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Qual è il tema principale del documento?
Agnoste il tempo?

Agnoste il principale?

Agnoste il principale?

Agnoste il principale?

Agnoste il principale?

Agnoste il principale?
Agnoste il principale?

Agnoste il principale?

Agnoste il principale?


Agnoste il principale?


Agnoste il principale?

Agnoste il principale?
Agnoste il principale?

Agnoste il principale


In [13]:
!cp -r gpt2_finetuned/ drive/MyDrive/gpt2_finetuned